In [ ]:
import os
import re
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

# =========================
# 0) ENV & CLIENT
# =========================
load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not OPENAI_API_KEY or not PINECONE_API_KEY:
    raise ValueError("❌ .env에 OPENAI_API_KEY / PINECONE_API_KEY 설정 필요")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME = "realestate2"
DEFAULT_NS = "__default2__"
index = pc.Index(INDEX_NAME)

EMBED_MODEL = "text-embedding-3-large"  # 3072
CHAT_MODEL  = "gpt-4.1-mini"

print("✅ 챗봇 준비 완료")
print(" - index:", INDEX_NAME)
print(" - namespace:", DEFAULT_NS)
print(" - embed:", EMBED_MODEL, "| chat:", CHAT_MODEL)

# =========================
# 1) PRIORITY MAP
# =========================
PRIORITY_MAP = {
    "주택임대차보호법": 1,
    "시행령": 2,
    "시행규칙/대법원규칙": 3,
    "민법": 4,
    "사례/판례/피해상담": 50,
}

def normalize(s: str) -> str:
    return re.sub(r"\s+", "", (s or "").strip())

def get_priority_from_md(md: dict) -> int:
    if not isinstance(md, dict):
        return 50
    p = md.get("priority")
    if p is not None:
        try:
            return int(p)
        except:
            pass

    lt = md.get("law_type") or md.get("category") or ""
    lt_n = normalize(str(lt))

    for k, v in PRIORITY_MAP.items():
        if normalize(k) == lt_n:
            return v

    # 느슨한 매칭
    if "주택임대차" in lt_n:
        return 1
    if "시행령" in lt_n:
        return 2
    if "시행규칙" in lt_n or "대법원규칙" in lt_n:
        return 3
    if "민법" in lt_n:
        return 4
    if "판례" in lt_n or "사례" in lt_n or "피해상담" in lt_n:
        return 50
    return 50

# =========================
# 2) Embedding
# =========================
def embed(text: str) -> list[float]:
    return client.embeddings.create(model=EMBED_MODEL, input=text).data[0].embedding

# =========================
# 3) Query intent -> quota
# =========================
def detect_intent(question: str) -> str:
    q = question.strip()

    # 원문/조문 그대로 출력 의도
    if re.search(r"(원문|그대로|전부|전체|조문|제\s*\d+\s*조|제\s*\d+\s*항|제\s*\d+\s*호)", q):
        return "verbatim"

    # 사례/판례 위주
    if re.search(r"(사례|판례|판결|결정|유사|비슷한|비교)", q):
        return "case"

    # 절차/방법 위주 (시행령/규칙이 도움 되는 경우 많음)
    if re.search(r"(어떻게|절차|방법|신청|요건|서류|가능|할\s*수|해야\s*해)", q):
        return "procedure"

    return "general"

def quota_for_intent(intent: str) -> dict:
    # priority: 1(법) 2(시행령) 3(규칙) 4(민법) 50(사례/판례)
    if intent == "verbatim":
        return {1: 7, 2: 2, 3: 1, 4: 0, 50: 0}
    if intent == "case":
        return {1: 2, 2: 1, 3: 1, 4: 0, 50: 6}
    if intent == "procedure":
        return {1: 3, 2: 3, 3: 3, 4: 1, 50: 0}
    return {1: 4, 2: 2, 3: 2, 4: 1, 50: 1}

# =========================
# 4) Keyword boost rerank
# =========================
STOPWORDS = {
    "그리고", "또는", "그래서", "그러면", "하지만", "때문에", "대한", "관련", "경우", "사실",
    "있는", "없는", "합니다", "해요", "되나요", "인가요", "어떻게", "무엇", "왜", "언제",
    "있다", "없다", "이다", "수", "것", "좀", "제가", "저는", "임차인", "임대인"
}

def extract_keywords_ko(text: str, max_kw: int = 8) -> list[str]:
    # 아주 단순한 토큰화: 한글/숫자 연속 2글자 이상
    toks = re.findall(r"[가-힣0-9]{2,}", text)
    uniq = []
    for t in toks:
        if t in STOPWORDS:
            continue
        if t not in uniq:
            uniq.append(t)
    return uniq[:max_kw]

def keyword_boost_score(question: str, evidence_text: str) -> float:
    # 질문의 키워드가 evidence에 포함되면 가중치
    kws = extract_keywords_ko(question, max_kw=8)
    if not kws:
        return 0.0
    hit = 0
    et = evidence_text
    for k in kws:
        if k in et:
            hit += 1
    # 너무 과하게 안 올리도록 0~0.12 사이
    return min(0.12, hit * 0.03)

# =========================
# 5) Retrieve + Priority + Keyword rerank + Quota
# =========================
def fetch_context_blocks(
    query: str,
    namespace: str = DEFAULT_NS,
    top_k: int = 60,
    max_blocks: int = 10,
    max_chars_per_block: int = 900,
):
    intent = detect_intent(query)
    per_priority_quota = quota_for_intent(intent)

    qvec = embed(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace,
    )

    items = []
    for m in (res.matches or []):
        md = m.metadata or {}
        text = md.get("text") or ""
        text = re.sub(r"\s+", " ", str(text)).strip()
        if len(text) < 30:
            continue

        p = get_priority_from_md(md)
        score = float(getattr(m, "score", 0.0))

        # 너무 길면 자르기(컨텍스트 폭발 방지)
        if max_chars_per_block and len(text) > max_chars_per_block:
            text = text[:max_chars_per_block].rstrip() + "…"

        # 키워드 가중치 추가
        boost = keyword_boost_score(query, text)
        adj_score = score + boost

        items.append({
            "id": getattr(m, "id", None),
            "score": score,
            "adj_score": adj_score,
            "boost": boost,
            "priority": p,
            "law_type": md.get("law_type") or md.get("category") or "",
            "article": md.get("article"),
            "chunk_id": md.get("chunk_id"),
            "page": md.get("page") or md.get("pageno") or md.get("page_no"),
            "source": md.get("source") or "unknown",
            "text": text,
        })

    if not items:
        return [], intent, per_priority_quota

    # 중복 제거(텍스트 기준)
    seen = set()
    dedup = []
    for it in items:
        key = re.sub(r"\s+", "", it["text"])
        if key in seen:
            continue
        seen.add(key)
        dedup.append(it)

    # 정렬: priority 우선 -> (가중치 적용 점수) 내림차순
    dedup.sort(key=lambda x: (x["priority"], -x["adj_score"]))

    # (source, article) 중복 억제
    sa_seen = set()
    sa_filtered = []
    for it in dedup:
        sa = (it["source"], it.get("article"))
        if it.get("article"):
            if sa in sa_seen:
                continue
            sa_seen.add(sa)
        sa_filtered.append(it)

    # quota로 선택
    selected = []
    counts = {k: 0 for k in per_priority_quota.keys()}
    for it in sa_filtered:
        p = it["priority"]
        if p not in per_priority_quota:
            continue
        if per_priority_quota[p] <= 0:
            continue
        if counts[p] >= per_priority_quota[p]:
            continue
        selected.append(it)
        counts[p] += 1
        if len(selected) >= max_blocks:
            break

    # 부족하면 남은 걸 채움
    if len(selected) < max_blocks:
        sel_keys = set(re.sub(r"\s+", "", x["text"]) for x in selected)
        for it in sa_filtered:
            if len(selected) >= max_blocks:
                break
            k = re.sub(r"\s+", "", it["text"])
            if k in sel_keys:
                continue
            selected.append(it)
            sel_keys.add(k)

    return selected[:max_blocks], intent, per_priority_quota

# =========================
# 6) LLM Prompt (LLM은 요지/답변만 작성)
# =========================
SYSTEM = """너는 '부동산/임대차 법령 근거 챗봇'이다.

절대 규칙:
- 너에게 주어지는 [근거 요약용 TEXT]에 포함된 내용만 바탕으로 작성한다.
- 근거에 없는 법리/절차/판례/추정/일반론을 추가하지 마라.
- 근거만으로 단정이 어렵다면 반드시 '근거 부족' 또는 '추가 확인 필요'라고 말하라.

출력 형식(반드시 지켜라):
1) [요지] : 근거 내용의 의미를 중복 없이 정리(새 정보 금지)
2) [답변] : 질문에 대해 요지를 쉬운 말로 설명(새 정보 금지)
3) [추가로 확인할 사실] : 필요한 질문 2~5개
"""

def format_evidence_block(blocks: list[dict]) -> str:
    # ✅ [근거 TEXT]는 코드가 그대로 출력 (LLM이 손대지 않음)
    lines = []
    for b in blocks:
        page = b.get("page")
        page_str = f", page={page}" if page is not None else ""
        lines.append(
            f'- (priority={b["priority"]}, law_type={b["law_type"]}, source={b["source"]}'
            f'{page_str}, article={b.get("article")}, chunk={b.get("chunk_id")}, '
            f'score={b["score"]:.3f}, boost={b["boost"]:.2f})\n'
            f'  "{b["text"]}"'
        )
    return "\n".join(lines)

def law_chat(question: str, namespace: str = DEFAULT_NS):
    blocks, intent, quota = fetch_context_blocks(
        question,
        namespace=namespace,
        top_k=60,
        max_blocks=10,
        max_chars_per_block=900,
    )
    if not blocks:
        return "근거 부족"

    evidence = format_evidence_block(blocks)

    # ✅ LLM에는 evidence를 "재인용"시키지 않고, 요지/답변만 쓰게 한다.
    # (그래도 근거 밖 말 금지 룰 유지)
    prompt = f"""[질문]
{question}

[근거 요약용 TEXT]
{evidence}

(주의) 위 TEXT를 그대로 재인용하거나 새로 만들어 쓰지 말고, 오직 요지/답변/추가확인만 작성해라.
"""

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ],
    )
    body = resp.choices[0].message.content.strip()

    # 최종 출력: ✅ 근거 TEXT는 코드가 그대로 붙임
    header = f"[의도분류] {intent} | [quota] {quota}\n"
    out = (
        header
        + "1) [근거 TEXT]\n"
        + evidence
        + "\n\n"
        + body
    )
    return out

# =========================
# 7) Interactive loop
# =========================
def run_chat(namespace: str = DEFAULT_NS):
    print("\n=== 🏠 RealEstate RAG Chat (근거 TEXT는 코드가 직접 출력) ===")
    print("명령어: /exit 종료 | /help 도움말 | /ns 현재 namespace 표시")
    print("그냥 질문을 입력하면, 근거 TEXT + (요지/답변/추가확인)을 출력합니다.\n")

    while True:
        try:
            q = input("You> ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\n종료합니다.")
            break

        if not q:
            continue

        if q.lower() in ["/exit", "exit", "quit", "/q"]:
            print("종료합니다.")
            break

        if q.lower() == "/help":
            print(" - 질문을 그냥 입력하세요.")
            print(" - /exit : 종료")
            print(" - /ns   : 현재 namespace 확인")
            continue

        if q.lower() == "/ns":
            print(f"(namespace = {namespace})")
            continue

        try:
            ans = law_chat(q, namespace=namespace)
            print("\nBot>\n" + ans + "\n")
        except Exception as e:
            print(f"\n[에러] {type(e).__name__}: {e}\n")

# ✅ 실행
run_chat()

✅ 챗봇 준비 완료
 - index: realestate2
 - namespace: __default2__
 - embed: text-embedding-3-large | chat: gpt-4.1-mini

=== 🏠 RealEstate RAG Chat (근거 TEXT 포함) ===
명령어: /exit 종료 | /help 도움말 | /ns 현재 namespace 표시
그냥 질문을 입력하면, 근거 TEXT를 같이 출력합니다.



You>  집주인이 보증금을 안 돌려주면 바로 소송 가능한가요?



Bot>
1) [근거 TEXT] :
- (priority=1, law_type=주택임대차보호법, source=주택임대차보호법(법률)(제21065호)(20260102).docx, article=제13조, chunk=0, score=0.547)
  "(「소액사건심판법」의 준용) 임차인이 임대인에 대하여 제기하는 보증금반환청구소송에 관하여는 「소액사건심판법」"

- (priority=4, law_type=사례/판례/피해상담, source=2025전세피해지원사례집.pdf, article=사례 4, chunk=0, score=0.589)
  "깡통주택 매수 문제 5 보증금반환소송 등 소송비용 문제 임대인이 보증금을 돌려줄 수 없으니 피해주택의 소유권을 대신 이전해 가라고 임대인을 대상으로 보증금반환소송 등을 진행하고자 하는데 소송비용이 부담됩니다. 합니다. 보증금으로 집을 매수해도 문제가 없을까요? l 임 대인이 피해주택 외 회수실익이 있는 재산과 소득이 없어 보증금반환이 불가한 상황이라면 l 국 토부 전세사기피해자(나목, 다목 포함) 또는 HUG 전세피해확인서를 발급받은 임차인에게는 대한변협 법률구조재단을 통해 소송대리 법률구조를 지원하고 있습니다. 전세피해 가구의 소득에 보증금으로 주택을 매수하여 소유권을 이전받는 방법을 고려해 볼 수 있습니다. 이 경우 지급명령 따라 중위소득 125%를 초과하는 피해자는 법률구조재단의 수행변호사와 매칭하여 수임료를 또는 보증금반환소송 및 경매를 통한 피해주택 강제처분 대비 시간과 비용을 절감할 수 있지만, 최대 250만원까지 지원하고 있으며, 중위소득 125% 이하인 피해자는 법률구조공단으로부터 보증금 대비 주택의 시세가 낮을 경우에는 손실이 발생할 수 있음을 유의하시기 바랍니다. 무료소송을 지원받을 수 있습니다. l 또 한, 압류 또는 가압류 등 권리제한이 있는 주택이라면 매수 이후 문제가 발생할 수 있으니, 법률 l 아 울러, 직접 변호사 또는 법무사를 통하여 집행권원을 확보하였거나, 나홀로소송을 진행하여 전문가와 상의한 이